In [1]:
import re
import json
import pprint
import os


Thunder_carrier_details = {"date": None,
                            "carrier_name": None,
                            "carrier_mc_no": None,
                            "noa_factoring_company_name": None,
                            "lor_factoring_company_name": None,
                            "noa_page_number": None,
                            "lor_page_number": None
}

def data_clean(text):
    try:
        strip_po = [i.strip() for i in text.split('\\n')]
        remove_space = [i for i in strip_po if i not in [' ', '']]
        remove_space_between = [i.split('  ') for i in remove_space]
        cleaned_data = list()
        count = 0
        for i in remove_space_between:
            cleaned_data.append([])
            for j in i:
                if j not in ['']:
                    cleaned_data[count].append(j.strip())
            count += 1
            
        return cleaned_data
    
    
    except Exception as e:
        print("kvt data clean exception")
        print(e)
        return None





f = open("./Thunder Carrier Services_1.txt", "r")
tmp = f.read()

tmp2 = data_clean(tmp)

def get_lines_between(origtmp, str1, str2):
    start_index = 0
    stop_index = 0
    for ele_nos, ele in enumerate(origtmp):
        for stchar in ele:
            if str1 in stchar.lower():
                start_index = ele_nos

            if str2 in stchar.lower():
                stop_index = ele_nos
                return origtmp[start_index:stop_index+1]
    return([])

def get_date(origtmp):
    '''
    Derives the Carrier name
    '''
    #Regex to find 3 digits followed by space, another 3 digits followed by space and 4 digits.
    regex_date1 = r'[\d]{1,2}\s?\/\s?[\d]{1,2}\s?\/\s?[\d]{4}|[A-Z]{1}[a-z]{3,7}\s[\d]{1,2}[,]{1}\s[\d]{4}'
    compiled_re1 = re.compile(regex_date1)
    
    #Get the relevant lines for search
    get_lines = get_lines_between(tmp2, "notice", "carrier")
    get_lines2 = get_lines_between(tmp2, "please", "mc")
    #Loop through the subset of lines and apply regex
    for line in get_lines:
        for wordstr in line:
            lc = compiled_re1.findall(wordstr)
            if len(lc) > 0:
                return(lc[0])
    else:
        for line in get_lines2:
            for wordstr in line:
                lc = compiled_re1.findall(wordstr)
                if len(lc) > 0:
                    return(lc[0])
                
    
    return('None')


def get_carrier_name(origtmp):
    '''
    Derives the Carrier name
    '''

    regex_car_name1 = r'[A-Za-z0-9&]+\s[A-Za-z0-9&,]+\s[A-Za-z0-9]+\s[llc|LLC]{3}|[A-Za-z0-9&]+\s[A-Za-z0-9,]+\s[LLC|llc]{3}'
    compiled_re1 = re.compile(regex_car_name1)
    
    regex_car_name2 = r'[A-Z]{1}[A-Za-z]+\s[A-Z]{1}[A-Za-z.]+\s[A-Z]{1}[A-Za-z]+|[A-Z]{1}[A-Za-z]+\s[A-Z]{1}[A-Za-z.]+'
    compiled_re2 = re.compile(regex_car_name2)
    
    regex_car_name3 = r'[A-Z]{1}[A-Za-z]{3,5}\s[A-Z]{1}[A-Za-z.-]{2,9}'
    compiled_re3 = re.compile(regex_car_name3)
    
    
       
    #Get the relevant lines for search
    get_lines = get_lines_between(tmp2, "notice", "re:")
    get_lines2 = get_lines_between(tmp2, "mc", "carrier acknowledgement")
    
    
    #Loop through the subset of lines and apply regex
    for line in get_lines:
        for wordstr in line:
            lc = compiled_re1.findall(wordstr)
            if len(lc) > 0:
                return(lc[0]) 
        else:
            lc = compiled_re3.findall(wordstr)
            if len(lc) > 0:
                return(lc[0])
        
    else: 
        for line in get_lines2:
            for wordstr in line:
                lc = compiled_re2.findall(wordstr)
                if len(lc) > 0:
                    return(lc[0])
                   
    return('None')



def get_carrier_mc_no(origtmp):
    '''
    Derives the MC code 
    '''
    #Regex to find letters MC no and the digits
    regex_ent_code = r'MC\sNO:\s\d+|MC#\d+'
    compiled_re = re.compile(regex_ent_code)
    
    #regex_ent_code2 = r'[3|1]{1}\d{6}'
    #compiled_re2 = re.compile(regex_ent_code2)
      
    #Get the relevant lines for search
    get_lines = get_lines_between(tmp2, "mc", "carrier acknowledgement")
    get_lines2 = get_lines_between(tmp2, "notice of assignment", "mc")
    
    #Loop through the subset of lines and apply regex
    for line in get_lines:
        for wordstr in line:
            lc = compiled_re.findall(wordstr)
            if len(lc) > 0:
                return(lc[0])
        #else:
         #   lc = compiled_re2.findall(wordstr)
          #  if len(lc) > 0:
           #     return(lc[0])    
           
    else: 
        for line in get_lines2:
            for wordstr in line:
                lc = compiled_re.findall(wordstr)
                if len(lc) > 0:
                    return(lc[0])
            #else:
             #   lc = compiled_re2.findall(wordstr)
              #  if len(lc) > 0:
               #     return(lc[0])    
    return('None')


def get_noa_factoring_company_name(origtmp):
    '''
    Derives factoring company name 
    '''
    #Regex to find 3 words all starting with a capital letter
    regex_noa_factoring = r'[A-Z]{1}[a-z]{4}\s[A-Z]{1}[a-z]{8}\s[A-Z]{1}[a-z]{3}'
    compiled_re = re.compile(regex_noa_factoring)
    
    #Get the relevant lines for search
    get_lines = get_lines_between(tmp2, "bank information for ach payment", "remittance")
    
    #Loop through the subset of lines and apply regex
    for line in get_lines:
        for wordstr in line:
            lc = compiled_re.findall(wordstr)
            if len(lc) > 0:
                return(lc[0])
    return('None')


def get_lor_factoring_company_name(origtmp):
    '''
    Derives lor_factoring company name 
    '''
    #Regex to find 3 words all starting with a capital letter
    regex_lor_factoring = r'[A-Z]{1}[a-z]{6}[\s][A-Z]{1}[a-z]{7}[\s][A-Z]{1}[a-z]{6}'
    compiled_re = re.compile(regex_lor_factoring)
    
    #Get the relevant lines for search
    get_lines = get_lines_between(tmp2, "all future payment", "dallas")
    
    #Loop through the subset of lines and apply regex
    for line in get_lines:
        for wordstr in line:
            lc = compiled_re.findall(wordstr)
            if len(lc) > 0:
                return(lc[0])
    return('None')
 

def get_noa_page_number(origtmp):
    '''
    Derives noa_page_number 
    '''
    text_1 = 'notice is to advise you'
    regex_noa_page_number = r'[A-Za-z]{6}\s[a-z]{2}\s[a-z]{2}\s[a-z]{6}\s[a-z]{3}'
    compiled_re1 = re.compile(regex_noa_page_number)
    
    get_lines = get_lines_between(tmp2, "attention accounts", "bank information for ach payment")
    
    for line in get_lines:
        for text_1 in line:
            lc = compiled_re1.findall(text_1)
            if len(lc) > 0:
                return('page 1')
    
    return('None')


def get_lor_page_number(origtmp):
    '''
    Derives lor_page_number 
    '''
    text_3 = 'termination of our UCC filing'
    text_4 = 'Please accept this letter as authorization'
    
    regex_lor_page_number_3 = r'[a-z]{11}\s[a-z]{2}\s[a-z]{3}\s[A-Z]{3}\s[a-z]{6}'
    compiled_re3 = re.compile(regex_lor_page_number_3)
    
    regex_lor_page_number_4 = r'[A-Za-z]{6}\s[a-z]{6}\s[a-z]{4}\s[a-z]{6}\s[a-z]{2}\s[a-z]{13}'
    compiled_re4 = re.compile(regex_lor_page_number_4)
    
    get_lines = get_lines_between(tmp2, "MC#", "please call me at")

    for line in get_lines:
        for text in line:
            lc = compiled_re3.findall(text_3)
            if len(lc) > 0:
                return('page 1')
        else:
            lc = compiled_re4.findall(text_4)
            if len(lc) > 0:
                return('page 1')
            
    return('None') 

def get_Thunder_carrier_info(tmp2):
    date = get_date(tmp2)
    carrier_name = get_carrier_name(tmp2)
    carrier_mc_no = get_carrier_mc_no(tmp2)
    noa_factoring_company_name = get_noa_factoring_company_name(tmp2)
    lor_factoring_company_name = get_lor_factoring_company_name(tmp2)
    noa_page_number = get_noa_page_number(tmp2)
    lor_page_number = get_lor_page_number(tmp2)
        
    Thunder_carrier_details["date"] = date
    Thunder_carrier_details["carrier_name"] = carrier_name
    Thunder_carrier_details["carrier_mc_no"] = carrier_mc_no
    Thunder_carrier_details["noa_factoring_company_name"] = noa_factoring_company_name
    Thunder_carrier_details["lor_factoring_company_name"] = lor_factoring_company_name
    Thunder_carrier_details["noa_page_number"] = noa_page_number
    Thunder_carrier_details["lor_page_number"] = lor_page_number

#get_Thunder_carrier_info(tmp2)

#pprint.pprint(Thunder_carrier_details)

for i in range(1, 26):
    thunder_carrier_details = {"date": None,
                                "carrier_name": None,
                                "carrier_mc_no": None,
                                "noa_factoring_company_name": None,
                                "lor_factoring_company_name": None,
                                "noa_page_number": None,
                                "lor_page_number": None 
    }
    filename = f'./Thunder Carrier Services_{i}.txt'
    with open(filename) as f:
        tmp2 = data_clean(f.read())
        filename_2 = f'Thunder Carrier Services_{i}'
        print(filename_2)
        
        get_Thunder_carrier_info(tmp2)
        pprint.pprint(Thunder_carrier_details)
        print('\n')


Thunder Carrier Services_1
{'carrier_mc_no': 'MC#1204528',
 'carrier_name': 'SIMA CARRIER',
 'date': '3/31/2021',
 'lor_factoring_company_name': 'None',
 'lor_page_number': 'None',
 'noa_factoring_company_name': 'First Tennessee Bank',
 'noa_page_number': 'page 1'}


Thunder Carrier Services_2
{'carrier_mc_no': 'MC#872796',
 'carrier_name': 'Sarvel Express-',
 'date': '1/22/2021',
 'lor_factoring_company_name': 'None',
 'lor_page_number': 'None',
 'noa_factoring_company_name': 'First Tennessee Bank',
 'noa_page_number': 'page 1'}


Thunder Carrier Services_3
{'carrier_mc_no': 'MC#930207',
 'carrier_name': 'Midwestern Express, LLC',
 'date': 'June 3, 2016',
 'lor_factoring_company_name': 'None',
 'lor_page_number': 'page 1',
 'noa_factoring_company_name': 'None',
 'noa_page_number': 'None'}


Thunder Carrier Services_4
{'carrier_mc_no': 'MC#872796',
 'carrier_name': 'Sarvel Express-',
 'date': '1/22/2021',
 'lor_factoring_company_name': 'None',
 'lor_page_number': 'None',
 'noa_factorin